In [2]:
from dotenv import load_dotenv

# 환경변수 로드 (.env)
load_dotenv()

True

In [1]:
import requests
import json
import os

# API 키와 엔드포인트 URL 설정
api_key = os.getenv('SERP_API_KEY')
url = "https://serpapi.com/search.json"

# 검색 매개변수 설정
params = {
    "api_key": api_key,
    "engine": "google_news",
    "q": "Tesla",
    "lang": "ko",
    "country": "kr"
}

# API 요청 보내기
response = requests.get(url, params=params)

# 응답 확인
if response.status_code == 200:
    # JSON 응답을 파이썬 딕셔너리로 변환
    data = json.loads(response.text)
    
    # 링크가 'N/A'가 아닌 기사만 필터링. 이 예시에선 forbes 기사만 필터링
    articles = []
    news_results = data.get("news_results", [])
    for article in news_results:
        link = article.get('link', 'N/A')
        # if link != 'N/A' and 'forbes' in link:
        #     articles.append(article)
        if 'forbes' in link:
            articles.append(article)
    
    # 필터링된 뉴스 기사 일부 출력
    for article in articles[:5]:
        print(f"제목: {article.get('title', 'N/A')}")
        print(f"링크: {article.get('link', 'N/A')}")
        print(f"출처: {article.get('source', 'N/A')}")
        print(f"게시 시간: {article.get('date', 'N/A')}")
        print("=====================================")
else:
    print(f"오류 발생: {response.status_code}")

제목: Tesla Earnings: Profits Fall For 4th Straight Quarter As Bottom Line Headaches Linger
링크: https://www.forbes.com/sites/dereksaul/2024/07/23/tesla-earnings-profits-fall-for-4th-straight-quarter-as-bottom-line-headaches-linger/
출처: {'name': 'Forbes', 'icon': 'https://lh3.googleusercontent.com/NIjzR9QHtZJllEvJIVkODXFQ7_rbx1BMBXx2rNUbttjuu9YWLIp2_juQaI9jJ2rDtzi6snbZ', 'authors': ['Derek Saul']}
게시 시간: 07/23/2024, 08:18 PM, +0000 UTC
제목: Tesla Slide Continues As Analysts Reality-Check Musk’s AI Hype That Has Overvalued Stock
링크: https://www.forbes.com/sites/roberthart/2024/07/24/tesla-slide-continues-as-analysts-reality-check-musks-ai-hype-that-has-overvalued-stock/
출처: {'name': 'Forbes', 'icon': 'https://lh3.googleusercontent.com/NIjzR9QHtZJllEvJIVkODXFQ7_rbx1BMBXx2rNUbttjuu9YWLIp2_juQaI9jJ2rDtzi6snbZ', 'authors': ['Robert Hart']}
게시 시간: 07/24/2024, 12:28 PM, +0000 UTC


In [3]:
# 전체 응답 구조 확인
print("API 응답 구조:")
print(json.dumps(data, indent=2, ensure_ascii=False))

# 필터링된 기사 수 출력
print(f"\n유효한 링크가 있는 기사 수: {len(articles)}")

API 응답 구조:
{
  "search_metadata": {
    "id": "66a37be5fa8675b86b94922d",
    "status": "Success",
    "json_endpoint": "https://serpapi.com/searches/2c9adf896e7dadd0/66a37be5fa8675b86b94922d.json",
    "created_at": "2024-07-26 10:35:17 UTC",
    "processed_at": "2024-07-26 10:35:17 UTC",
    "google_news_url": "https://news.google.com/search?q=Tesla&gl=US",
    "raw_html_file": "https://serpapi.com/searches/2c9adf896e7dadd0/66a37be5fa8675b86b94922d.html",
    "total_time_taken": 0.79
  },
  "search_parameters": {
    "engine": "google_news",
    "gl": "us",
    "q": "Tesla"
  },
  "news_results": [
    {
      "position": 1,
      "title": "Tesla's second-quarter earnings report",
      "stories": [
        {
          "position": 1,
          "title": "Tesla stock slides after company reports mixed Q2 results, says 'affordable' EVs on track for 2025 debut",
          "source": {
            "name": "Yahoo Finance",
            "icon": "https://lh3.googleusercontent.com/L57n4HUzUS8lN

In [7]:
articles[0].get('link', 'N/A')

'https://www.forbes.com/sites/dereksaul/2024/07/23/tesla-earnings-profits-fall-for-4th-straight-quarter-as-bottom-line-headaches-linger/'

In [8]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# 당연하게도 뉴스 사이트 별로 파싱해야 할 부분이 다름
loader = WebBaseLoader(
    web_paths=[articles[0].get('link', 'N/A')],
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            name=["div"],  # 태그
            attrs={"class": ["article-body fs-article fs-responsive-text current-article article-topline"]},  # 태그의 클래스 값
        )
    ),
)
data = loader.load()

print(data[0].page_content)


Share to FacebookShare to TwitterShare to LinkedinTopline
Tesla stock faltered after its Tuesday afternoon earnings report, despite better-than-forecasted sales, as the company’s extended profit growth slump continues to eat into its valuation.
Tesla reported Q2 vehicle deliveries above expectations earlier this month. AFP via Getty Images

Key Facts


Tesla raked in $0.52 of earnings per share (EPS) in the second quarter, falling well short of consensus analyst projections of $0.61.



That’s 43% below Q2 2023’s $0.91 EPS, marking the fourth consecutive quarter of negative profit growth for the car maker.



Yet Tesla’s $25.5 billion of sales topped estimates of $24.5 billion, up 2% year-over-year.



Shares of the ever-volatile Tesla fell about 3% after the mixed earnings release.


Get Forbes Breaking News Text Alerts: We’re launching text message alerts so you'll always know the biggest stories shaping the day’s headlines. Text “Alerts” to (201) 335-0739 or sign up here.


Key Back

In [9]:
from langchain_community.chat_models import ChatOllama

eeve = ChatOllama(model="EEVE-Korean-Instruct-10.8B-v1.0:latest")

In [10]:
prompt = f"""
당신은 금융 분야의 요약 전문가입니다. 

아래의 금융 관련 텍스트를 읽고, 다음 지침에 따라 요약해 주세요:
1. 전체 내용을 50단어 이내로 요약한 'summary' 필드를 작성해 주세요.
2. 핵심 내용을 5-10개 간결하게 정리해서 'key_points' 필드에 작성해 주세요.
3. 각 key_points 내용은 한 문장으로, 20단어를 넘지 않도록 해주세요.
4. 텍스트의 핵심 키워드와 관련된 단어나 개념을 10개 내외로 'tags' 필드에 추가해 주세요.
5. 전문 용어가 있다면 간단히 설명을 덧붙여 주세요.
6. 숫자나 통계가 있다면 반드시 포함시켜 주세요.
7. 요약은 객관적이고 중립적인 톤을 유지해 주세요.

응답은 반드시 아래의 JSON 형식을 따라 주세요: 
{{
    "summary": "전체 내용 요약 (50단어 이내)",
    "key_points": [
        "핵심 포인트 1 (20단어 이내)",
        "핵심 포인트 2 (20단어 이내)",
        "핵심 포인트 3 (20단어 이내)",
        "핵심 포인트 4 (20단어 이내)",
        "핵심 포인트 5 (20단어 이내)",
        ...
    ],
    "tags": ["태그1", "태그2", "태그3", ...]
}}

텍스트: 
{data[0].page_content}

위 지침에 따라 JSON 형식으로 요약해 주세요. 
"""
response = eeve.invoke(prompt)

In [12]:
# JSON 파싱
import json
parsed_response = json.loads(response.content)
parsed_response

{'summary': '테슬라, 이익 성장 정체에도 불구하고 예상보다 나은 판매량에도 불구하고 화요일 오후 실적 발표 후 주가가 하락했습니다.',
 'key_points': ['테슬라는 Q2에 주당 0.52달러의 이익을 기록했으며, 이는 예상치인 0.61달러를 크게 하회함',
  '이는 자동차 제조업체에게 네 번째 연속적인 이익 성장률 하락을 의미하며, 전년 동기 대비 43% 감소함',
  '그러나 테슬라의 매출은 255억 달러로 추정치를 상회했으며, 지난해 같은 기간보다 2% 증가함',
  '혼합된 실적 발표 후 주가는 약 3% 하락함',
  '테슬라는 Q2에 44만 4천 대의 차량을 인도했는데, 이는 전년 동기 대비 5% 감소했지만 예상치보다는 나은 결과임'],
 'tags': ['테슬라', '주식', '실적', '이익 성장', '전기차', '엘론 머스크']}